In [1]:
async def coroutine_1():
    return 1

c = coroutine_1()

type(c)

coroutine

`await asyncio.sleep(0)`  command for yielding control to the event loop so that other tasks can run

Storing tasks in a list allows a way to await their completion or check their status using `t.done()`

The ` while True` loop continuously checks the status of tasks in the tasks list. It filters out completed tasks and then checks if there are any tasks left. If there are no tasks left (i.e., all tasks are done), it returns from the main function, effectively ending the program.

The crucial part is await tasks[0], which means the program will pause at this point until the first task in the list (tasks[0]) is completed. This ensures that the loop doesn't proceed until at least one task has finished. 

If you remove this line, the loop won't wait for any task to complete, and it will keep checking the task statuses in a tight loop without giving other tasks a chance to execute. This can lead to a busy-waiting scenario, where the program consumes CPU resources without making progress, and the loop never exits.


In [1]:
import asyncio

# Define an asynchronous function 'counter' that takes a 'name' parameter.
async def counter(name: str):
    # Loop from 0 to 9 (inclusive).
    for i in range(0, 5):
        # Print the name and current value of 'i'.
        print(f"{name}: {i!s}")
        # Pause the execution of this coroutine to allow other tasks to run.
        await asyncio.sleep(0)  # This line will pause the current task and allow other tasks to be executed.

# Define the main asynchronous function.
async def main():
    tasks = []  # Create an empty list to store tasks.

    # Create three tasks and append them to the 'tasks' list.
    for n in range(0, 3):
        tasks.append(asyncio.create_task(counter(f"task{n}")))

    while True:
        # Filter out tasks that have completed (done).
        tasks = [t for t in tasks if not t.done()]

        print('len(tasks)', len(tasks))
        # If all tasks are done, return from the 'main' function.
        if len(tasks) == 0:
            return

        # Wait for the completion of the first task in the 'tasks' list.
        await tasks[0] # without this line there is no stop 

# Run the 'main' function using the asyncio event loop.
# asyncio.run(main())
await main()

len(tasks) 3
task0: 0
task1: 0
task2: 0
task0: 1
task1: 1
task2: 1
task0: 2
task1: 2
task2: 2
task0: 3
task1: 3
task2: 3
task0: 4
task1: 4
task2: 4
len(tasks) 0


To run `async def main()` above:

in a python script you would do  `asyncio.run(main())` such as in scripts/monitor.py

```python
import asyncio

asyncio.run(main())
```

jupyter (IPython ≥ 7.0) is already running an event loop and `RuntimeError: asyncio.run() cannot be called from a running event loop`

so in jupyter you just run:

`await main()`